# Fine-tuning do BERTimbau com o dataset COVID.BR
Giulia Chimini Stefainski, Leonardo Azzi Martins, Matheus de Moraes Costa

---

**Objetivo:** realizar fine-tuning do modelo BERTimbau com o dataset COVID.BR, para classificação de desinformação em mensagens de texto do WhatsApp.

**Referências:**
- Laboratório 6 - Fine Tunnig BERT (Encoder), Prof. Dennis e Rafael Oleques
- [GitHub - Performing fine-tuning on BERT and BERTimbau models for text classification in Portuguese using the News Dataset in Portuguese](https://github.com/szanara/Bert-Bertimbau/blob/main/Bert_e_Bertimbal.ipynb)

![](https://www.aprendizartificial.com/wp-content/uploads/2024/02/JZ7Hynh.png)

# Setup

In [0]:
%pip install pandas==1.5.3 transformers==4.50.2 datasets==3.5.0 torch==2.6.0+cu124 scikit-learn==1.4.2 evaluate==0.4.3 seaborn==0.13.2 imblearn accelerate==1.5.2 emoji

In [0]:
%restart_python

In [0]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

# Preparação de dados
Carrega o dataset a ser utilizado para fine-tuning e seleciona os atributos mais relevantes.

Faz o download do dataset anotado no diretório ./data

In [0]:
import os

if not os.path.exists('./data/covidbr_labeled.csv'):
  %mkdir data
  %curl -L -o ./data/covidbr_labeled.csv https://zenodo.org/records/5193932/files/covidbr_labeled.csv
else:
    print("File already exists. Skipping download.")

In [0]:
original_dataset_df = pd.read_csv('./data/covidbr_labeled.csv')
original_dataset_df

In [0]:
dataset_df = original_dataset_df[["text", "misinformation"]]
dataset_df

# Análise exploratória de dados

O objetivo é entender melhor e sumarizar as características dos dados, analisando quantidade e tipos de atributos, verificando distribuição do atributo alvo, identificando padrões e anomalias, removendo atributos que pareçam irrelevantes ou problemáticos, etc. Utilize gráficos e sumarizações estatísticas para a EDA. Verifique potenciais problemas nos dados, como por exemplo, a necessidade de normalizar os atributos, balancear classes, ou remover instâncias ou atributos por inconsistências nos dados.

- P1. Qual a quantidade e tipos de atributos? Existem inconsistências?
  - Quais são os atributos disponíveis?
  - Existem inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)
  - Existem atributos que necessitam ser removidos ou transformados?
- P2. Qual a distribuição do atributo alvo?
  - Quais são as classes alvo? Qual a distribuição entre as classes? Está balanceada ou desbalanceada?


## P1. Qual a quantidade e tipos de atributos? Existem inconsistências?

### 1.1 Quais são os atributos disponíveis?

In [0]:
dataset_df.info()

### 1.2 Existem inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)

#### 1.2.1 Existem valores nulos?

In [0]:
dataset_df[dataset_df.isnull().any(axis=1)]

Remove instância com texto nulo, pois é irrelevante para o treinamento

In [0]:
dataset_df = dataset_df.dropna()
dataset_df.isnull().any()

#### 1.2.2 Existem textos que começam com URLs e podem ser removidos?

##### Busca por textos que começam com URLs

Busca instâncias de text onde começa com uma URL. Conforme Martins et al. 2021, estas instâncias podem dificultar a classificação, resultando em um ganho de aprox. 10% em F1-score ao remover estas instâncias.

In [0]:
dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)]

In [0]:
dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)]

In [0]:
dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)].count()

#### 1.2.3 Existe o mesmo dataset filtrado conforme Martins et al. (2021)?

##### Reproduz o notebook de Martins et al. (2021)

In [0]:
import re
dataset_df['cleanLinks'] = dataset_df['text'].apply(lambda x: re.split(r'http:\/\/.*', str(x))[0])

In [0]:
dataset_df['cleanLinks']

In [0]:
dataset_df[dataset_df['cleanLinks'] != '' ].shape

#### 1.2.4 Existem textos que contém URLs e podem ser removidos?

##### Busca por textos que contém URLs e analisa a possibilidade de removê-los

In [0]:
url_pattern = r'\b(?:https?://|www\.)\S+\b|\b\S+\.(com|br)\b'

no_url_df = dataset_df.copy()
no_url_df = no_url_df[~no_url_df['text'].str.contains(url_pattern, regex=True, flags=re.IGNORECASE)]
no_url_df

In [0]:
no_url_df.shape

#### 1.2.5 Existem textos com emojis que podem ser transformados ou removidos?

##### Busca mensagens compostas por emojis
- `emoji_count`: conta a quantidade de emojis em 'text'
- `char_count`: conta a quantidade de caracteres em 'text'
- `emoji_ratio`: calcula a taxa de emojis por mensagem

In [0]:
import emoji

def count_emojis(text):
    return sum(1 for char in text if char in emoji.EMOJI_DATA)

def char_count(text):
    return len(text)

def word_count(text):
    return len(text.split())

dataset_df['emoji_count'] = dataset_df['text'].apply(count_emojis)
dataset_df['char_count'] = dataset_df['text'].apply(char_count)
dataset_df['word_count'] = dataset_df['text'].apply(word_count)

def emoji_ratio(text):
    return count_emojis(text) / char_count(text) if char_count(text) > 0 else 0

dataset_df['emoji_ratio'] = dataset_df['text'].apply(emoji_ratio)

dataset_df

Verifica as instâncias com maior emoji_ratio

In [0]:
display(dataset_df.sort_values(by='emoji_ratio', ascending=False).reset_index(drop=True))

Existe alguma relação entre a taxa de emojis e o atributo preditivo?

In [0]:
dataset_df[dataset_df['misinformation'] == 0]['emoji_ratio'].describe()

In [0]:
dataset_df[dataset_df['misinformation'] == 1]['emoji_ratio'].describe()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6, 4))
sns.boxplot(x='misinformation', y='emoji_ratio', data=dataset_df, palette=['#377eb8', '#e41a1c'])
plt.title('Emoji Ratio vs Misinformation')
plt.xlabel('Misinformation')
plt.ylabel('Emoji Ratio')
plt.show()

Verifica as instâncias com maior emoji_count

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(dataset_df['emoji_count'], edgecolor='black')
plt.title('Histogram of emoji_count')
plt.xlabel('emoji_count')
plt.ylabel('Frequency')
plt.show()

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(dataset_df['emoji_count'], dataset_df['char_count'], alpha=0.5)
plt.title('Scatter Plot of Emoji Count vs Character Count')
plt.xlabel('Emoji Count')
plt.ylabel('Character Count')
plt.show()

In [0]:
display(dataset_df.sort_values(by='emoji_count', ascending=False).reset_index(drop=True))

Não existem textos com emoji ratio maior que ~0.02. Portanto, não precisam ser tratados.

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(dataset_df['emoji_ratio'], edgecolor='black')
plt.title('Histogram of moji_ratio')
plt.xlabel('emoji_ratio')
plt.ylabel('Frequency')

plt.show()

Existem textos com poucas palavras significativas?

In [0]:
display(dataset_df.sort_values(by='word_count', ascending=True).reset_index(drop=True))

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(dataset_df['word_count'], edgecolor='black')
plt.title('Histogram of Word Count')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.show()

In [0]:
dataset_df.info()

## P2. Qual a distribuição do atributo alvo?

### Experimento 1

In [0]:
import seaborn as sns

series = dataset_df['misinformation'].value_counts()

print(series)

fig = plt.figure(figsize=(5, 3))

sns.countplot(x=dataset_df['misinformation'], data = dataset_df,
              hue='misinformation', palette=['#377eb8', '#e41a1c'],
              order=dataset_df['misinformation'].value_counts().index
)

Isto indica que o dataset está desbalanceado, fator que pode enviesar o treinamento.

### Experimento 3

In [0]:
import seaborn as sns

series = no_url_df['misinformation'].value_counts()

print(series)

fig = plt.figure(figsize=(5, 3))

sns.countplot(x=no_url_df['misinformation'], data = no_url_df,
              hue='misinformation', palette=['#377eb8', '#e41a1c'],
              order=dataset_df['misinformation'].value_counts().index
)

# Pré-processamento

In [0]:
  # Configurações iniciais
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"Usando o dispositivo: {device}")

## Limpeza

In [0]:
dataset_df.info()

Dataset removendo instâncias onde o texto contém URLs em seu início

In [0]:
no_start_url_df = dataset_df[~dataset_df['text'].str.contains(r'^(http|www)', na=False)].reset_index(drop=True)
no_start_url_df.info()

Dataset removendo todas as URLs do texto

In [0]:
no_url_df.info()

## Labels
O HuggingFace Trainer utiliza o rótulo labels para identificar os rótulos no treinamento. Renomeando a coluna alvo para 'labels'

In [0]:
dataset_df = dataset_df.rename(columns={'misinformation': 'labels'})
no_start_url_df = no_start_url_df.rename(columns={'misinformation': 'labels'})
no_url_df = no_url_df.rename(columns={'misinformation': 'labels'})

## Tokenização

Carrega o tokenizador para `bert-base-portuguese-cased` (BERTimbau)

In [0]:
from transformers import AutoTokenizer  # Or BertTokenizer

hf_model_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(hf_model_name, do_lower_case=False)

Criamos uma função de tokenização, que será utilizada para tokenizar cada valor de um Pandas DataFrame em forma de função de mapeamento.

In [0]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

In [0]:
def tokenize(df):
  dataset = Dataset.from_pandas(df)
  dataset_tk = dataset.map(tokenize_function, batched=True, remove_columns=['text']) #'__index_level_0__'
  return dataset_tk

## Balanceamento de classes

Dado que o dataset tem sua classe misinformation desbalanceada, utilizou-se o método de cálculo de class_weights, que atribui pesos na função loss do treinador para 'compensar' o desbalanceamento.

"If "balanced", class weights will be given by `n_samples / (n_classes * np.bincount(y=labels))`. If a dictionary is given, keys are classes and values are corresponding class weights. If None is given, the class weights will be uniform."

Referências:
- https://medium.com/@heyamit10/fine-tuning-bert-for-classification-a-practical-guide-b8c1c56f252c
- https://discuss.huggingface.co/t/class-weights-for-bertforsequenceclassification/1674

get_class_weights(df):
- Cria uma instância do CrossEntropyLoss com os pesos calculados
- Recria a classe WeightedTrainer para 'sobrescrever' a classe original no HuggingFace Trainer, utilizada a computação do loss ponderada configurada acima.

In [0]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

def get_class_weights(df):
  labels = df["labels"]

  class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)

  class_weights = torch.tensor(class_weights, dtype=torch.float)

  print(class_weights)

  loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

  class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss
      
  return WeightedTrainer

In [0]:
get_class_weights(dataset_df)

# Fine-tuning com validação cruzada

## Configuração

### Dados

In [0]:
dataset_df = dataset_df[['labels', 'text']]
dataset_df

In [0]:
no_start_url_df = no_start_url_df[['labels', 'text']]
no_start_url_df

In [0]:
no_url_df = no_url_df[['labels', 'text']]
no_url_df

### Treinador

In [0]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os
import mlflow

In [0]:
!mkdir -p models/covidbr_pt

Função para calcular métricas com as predições e retornar como um dicionário

In [0]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # print(preds)
    prec = precision_score(labels, preds, average='weighted')
    rec = recall_score(labels, preds, average='weighted')
    f1_mi = f1_score(labels, preds, average='micro')
    f1_ma = f1_score(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    fp_rate = 1 - prec  # False positive rate
    return {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1-macro': f1_ma,
        'f1-micro': f1_mi,
        'fpr': fp_rate
    }

Função de treinamento do BERT

In [0]:
def train_bert(train_data, val_data, run_name):
  # MLFlow utilizado para rastrear o treinamento e seus artefatos
  # Utilizado com Databricks
  mlflow.end_run()
  with mlflow.start_run(run_name=f"{experiment_name}_{run_name}"):
    model = AutoModelForSequenceClassification.from_pretrained(hf_model_name,
                                                                num_labels=2)
    model.to(device)

    # Hiperparâmetros
    batch_size = 8
    epochs = 15
    learning_rate = 1e-06
    steps_per_epoch = round(len(train_data) / batch_size)
    output_dir=f'/tmp/bert_output_{experiment_name}_{run_name}'
    
    # Argumentos de treinamento
    training_args = TrainingArguments(
      output_dir=output_dir,
      overwrite_output_dir=True,
      eval_strategy='epoch',
      save_strategy='epoch',
      save_only_model=True,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      logging_steps=20,
      report_to="mlflow",
      learning_rate=learning_rate,
      num_train_epochs=epochs,
      load_best_model_at_end=False,
    )

    print(training_args)

    # Calcula os pesos de balanceamento para os dados de treino
    WeightedTrainer = get_class_weights(train_data)

    trainer = WeightedTrainer(
      model=model,
      args=training_args,
      train_dataset=train_data,
      eval_dataset=val_data,
      compute_metrics=compute_metrics,
    )

    # O treinamento acontece aqui
    trainer.train()

    output_dir = 'models/covidbr_pt'

    # Salva artefatos de treinamento
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    model.save_pretrained(output_dir, safe_serialization=False)

    print("Arquivos salvos:")
    print(os.listdir(output_dir))
    
  mlflow.end_run()

  return trainer

### K-Fold Cross Validation
https://www.philschmid.de/k-fold-as-cross-validation-with-a-bert-text-classification-example

Função para treinamento com K-Fold Cross Validation, onde todas as porções dos dados podem ser utilizadas como teste uma vez.

In [0]:
from sklearn.model_selection import cross_validate, KFold

# Realiza a validação cruzada de um modelo, dado o número de folds e a métrica
def cross_validate_model(data, folds, metrics, metrics_df, rand_state):

  # Avalia o modelo em k folds
  cv = KFold(n_splits=folds, random_state=rand_state, shuffle=True)

  for fold, (train_idx, test_idx) in enumerate(cv.split(data)):
    print(f"=== Fold {fold + 1} ===")

    print(f"Fold {fold + 1}")

    # Divide os dados em treino e validação
    train_df = data.iloc[train_idx]
    test_df = data.iloc[test_idx]

    # Tokeniza os dados de treino e validação
    train_tk = tokenize(train_df)
    test_tk = tokenize(test_df)

    # Cria o nome do modelo chama o treinador
    run_name = f'fold_{fold}'
    trainer = train_bert(train_tk, test_tk, run_name)

    eval_result = trainer.evaluate()

    # Organiza os resultados em um DataFrame
    fold_scores = {'Fold': fold}
    fold_scores.update({metric: eval_result[f'eval_{metric}'] for metric in metrics})
    fold_df = pd.DataFrame(fold_scores, index=[0])
    metrics_df = pd.concat([metrics_df, fold_df], ignore_index=True)

  return eval_result, trainer, metrics_df

## Experimento 1: dataset original

In [0]:
experiment_name = "covidbr_bert"

In [0]:
# Hiperparâmetros para validação cruzada
k_folds = 5
random_state = 42
metrics = ['accuracy', 'precision', 'recall', 'f1-macro', 'f1-micro', 'fpr']

# DataFrame para armazenar os resultados das métricas
metrics_df = pd.DataFrame(columns=['Fold']+metrics)

result, trainer, metrics_df = cross_validate_model(dataset_df, k_folds, metrics, metrics_df, random_state)

metrics_df

In [0]:
metrics_df.to_csv(f'{experiment_name}_metrics_kfold.csv')

## Experimento 2: removendo textos que começam com URL

In [0]:
experiment_name = "covidbr_nostarturl_bert"

In [0]:
# Hiperparâmetros para validação cruzada
k_folds = 5
random_state = 42
metrics = ['accuracy', 'precision', 'recall', 'f1-macro', 'f1-micro', 'fpr']

# DataFrame para armazenar os resultados das métricas
metrics_df = pd.DataFrame(columns=['Fold']+metrics)

result, trainer, metrics_df = cross_validate_model(no_start_url_df, k_folds, metrics, metrics_df, random_state)

metrics_df

Salva as métricas do K-Fold em .csv

In [0]:
metrics_df.to_csv(f'{experiment_name}_metrics_kfold.csv')

## Experimento 3: removendo todos os textos que contém URLs

In [0]:
experiment_name = "covidbr_nourl_bert"

In [0]:
# Hiperparâmetros para validação cruzada
k_folds = 5
random_state = 42
metrics = ['accuracy', 'precision', 'recall', 'f1-macro', 'f1-micro', 'fpr']

# DataFrame para armazenar os resultados das métricas
metrics_df = pd.DataFrame(columns=['Fold']+metrics)

result, trainer, metrics_df = cross_validate_model(no_url_df, k_folds, metrics, metrics_df, random_state)

metrics_df

In [0]:
metrics_df.to_csv(f'{experiment_name}_metrics_kfold.csv')